<a href="https://colab.research.google.com/github/Sivasubramaniansivanayagam/Pyspark-Diabetes_Prediction/blob/main/Pyspark_Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run Spark Session

In [ ]:
#install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 64.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=0684481c629f1fb9f7100a72bf407ef1036ec1efa200f2bad72e28f4136427a9
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.mllib import classification
import numpy as np
from sklearn.metrics import plot_confusion_matrix

In [ ]:
spark=SparkSession.builder.appName("test").getOrCreate()
spark

# TASK 2: Clone & Explore dataset

In [ ]:
#clone the diabetes dataset from the github repository
! git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
#check if the dataset exists

! ls diabetes_dataset

diabetes.csv  new_test.csv


In [ ]:
#create spark dataframe
df=spark.read.csv("/content/diabetes_dataset/diabetes.csv",header=True,inferSchema=True)

In [ ]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [ ]:
#print the schema
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [ ]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#count the total no. of diabetic and non-diabetic class

print(df.count(),(len(df.columns)))
df.groupBy("Outcome").count().show()

2000 9
+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [ ]:
#get the summary statistics
df.describe().show()

# print(np.percentile(np.array(df.select("Pregnancies"),25,axis=1)))

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [ ]:
from pyspark.sql.window import Window
df.select("Glucose","BloodPressure",\
f.percent_rank().over(Window.partitionBy("Glucose").orderBy(f.desc("BloodPressure"))).alias("Rank")).show()

+-------+-------------+------------------+
|Glucose|BloodPressure|              Rank|
+-------+-------------+------------------+
|      0|           80|               0.0|
|      0|           80|               0.0|
|      0|           80|               0.0|
|      0|           74|              0.25|
|      0|           74|              0.25|
|      0|           68|0.4166666666666667|
|      0|           68|0.4166666666666667|
|      0|           68|0.4166666666666667|
|      0|           68|0.4166666666666667|
|      0|           68|0.4166666666666667|
|      0|           68|0.4166666666666667|
|      0|           48|0.9166666666666666|
|      0|           48|0.9166666666666666|
|     44|           62|               0.0|
|     44|           62|               0.0|
|     56|           56|               0.0|
|     56|           56|               0.0|
|     56|           56|               0.0|
|     57|           80|               0.0|
|     57|           80|               0.0|
+-------+--

In [ ]:
df.select("Glucose","BloodPressure",\
f.dense_rank().over(Window.partitionBy("Glucose").orderBy(f.desc("BloodPressure"))).alias("Rank")).show()

+-------+-------------+----+
|Glucose|BloodPressure|Rank|
+-------+-------------+----+
|      0|           80|   1|
|      0|           80|   1|
|      0|           80|   1|
|      0|           74|   2|
|      0|           74|   2|
|      0|           68|   3|
|      0|           68|   3|
|      0|           68|   3|
|      0|           68|   3|
|      0|           68|   3|
|      0|           68|   3|
|      0|           48|   4|
|      0|           48|   4|
|     44|           62|   1|
|     44|           62|   1|
|     56|           56|   1|
|     56|           56|   1|
|     56|           56|   1|
|     57|           80|   1|
|     57|           80|   1|
+-------+-------------+----+
only showing top 20 rows



In [ ]:
np.array(df.select("Pregnancies"))

array(DataFrame[Pregnancies: int], dtype=object)

# TASK 3: Data Cleaning & Preparation

In [ ]:
#check for null values
# Method 1
df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [ ]:
# Method 2 to check null values

for col in df.columns:
  print(col,df[df[col].isNull()].count())


Pregnancies 0
Glucose 0
BloodPressure 0
SkinThickness 0
Insulin 0
BMI 0
DiabetesPedigreeFunction 0
Age 0
Outcome 0


In [ ]:
#look for the unnecessary values present

df.select([f.count(f.when(f.col(c)==0,c)).alias(c) for c in df.columns]).show()


+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|        301|     13|           90|          573|    956| 28|                       0|  0|   1316|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [ ]:
# Method 2
for i in df.columns:
  print(i,df[df[i]==0].count())

Pregnancies 0
Glucose 0
BloodPressure 0
SkinThickness 0
Insulin 0
BMI 0
DiabetesPedigreeFunction 0
Age 0
Outcome 1316


In [ ]:
#calculate and replace the unnecessary values by the mean value

for i in df.columns:
  data=df.agg({i:"mean"}).first()
  print(data)


Row(avg(Pregnancies)=4.155)
Row(avg(Glucose)=121.969)
Row(avg(BloodPressure)=72.2505)
Row(avg(SkinThickness)=26.665)
Row(avg(Insulin)=118.494)
Row(avg(BMI)=32.640999999999984)
Row(avg(DiabetesPedigreeFunction)=0.47092999999999974)
Row(avg(Age)=33.0905)
Row(avg(Outcome)=0.342)


In [ ]:
for i in df.columns:
  data=df.agg({i:"mean"}).first()[0]
  print(f'mean for {i} is {int(data)}')
  print(df[i])

mean for Pregnancies is 4
Column<'Pregnancies'>
mean for Glucose is 121
Column<'Glucose'>
mean for BloodPressure is 72
Column<'BloodPressure'>
mean for SkinThickness is 26
Column<'SkinThickness'>
mean for Insulin is 118
Column<'Insulin'>
mean for BMI is 32
Column<'BMI'>
mean for DiabetesPedigreeFunction is 0
Column<'DiabetesPedigreeFunction'>
mean for Age is 33
Column<'Age'>
mean for Outcome is 0
Column<'Outcome'>


In [ ]:
for i in df.columns:
  data=df.agg({i:"mean"}).first()[0]
  print(f'mean for {i} is {int(data)}')
  df=df.withColumn(i,f.when(df[i]==0,int(data)).otherwise(df[i]))

mean for Pregnancies is 3
mean for Glucose is 121
mean for BloodPressure is 69
mean for SkinThickness is 20
mean for Insulin is 80
mean for BMI is 32
mean for DiabetesPedigreeFunction is 0
mean for Age is 33
mean for Outcome is 0


In [ ]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          3|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          3|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          3|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          3|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


# TASK 4: Correlation Analysis & Feature Selection

In [ ]:
#find the correlation among the set of input & output variables

import seaborn as sns

len(df.columns)
inp=df.select(df.columns[0:8])

for i in df.columns:
  print(f'correlation of {i} is {df.stat.corr("Outcome",i)}')


correlation of Pregnancies is 0.24790480177711924
correlation of Glucose is 0.48796646527321064
correlation of BloodPressure is 0.17171333286446713
correlation of SkinThickness is 0.1659010662889893
correlation of Insulin is 0.1711763270226193
correlation of BMI is 0.2827927569760082
correlation of DiabetesPedigreeFunction is 0.1554590791569403
correlation of Age is 0.23650924717620253
correlation of Outcome is 1.0


In [ ]:
lis=[]
for i in df.columns:
  lis.append(i)

print(lis[0:8])
lis


['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
#feature selection

from pyspark.ml.feature import VectorAssembler

assembler=VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'],
                          outputCol="features")
output_data=assembler.transform(df)


In [ ]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          3|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[3.0,84.0,82.0,31...|
|          3|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[3.0,145.0,69.0,2...|
|          3|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[3.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          3|    173|           

In [ ]:
#print the schema
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#display dataframe
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          3|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[3.0,84.0,82.0,31...|
|          3|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[3.0,145.0,69.0,2...|
|          3|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[3.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          3|    173|           

# TASK 5: Split Dataset & Build the Model

In [ ]:
#create final data
from pyspark.ml.classification import LogisticRegression
finaldata=output_data.select("features","Outcome")

In [ ]:
#print schema of final data
finaldata.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#split the dataset ; build the model
train,test=finaldata.randomSplit([0.7,0.3])
models=LogisticRegression(labelCol="Outcome")
model=models.fit(train)

In [ ]:
#summary of the model
summary=model.summary
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|            Outcome|         prediction|
+-------+-------------------+-------------------+
|  count|               1398|               1398|
|   mean|0.34763948497854075| 0.2675250357653791|
| stddev|0.47639123787597426|0.44282689588185026|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# TASK 6: Evaluate and Save the Model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

prediction=model.evaluate(test)

In [ ]:
prediction.predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,71.0,48.0,18...|      0|[4.19728856142015...|[0.98518644928897...|       0.0|
|[1.0,71.0,48.0,18...|      0|[4.19728856142015...|[0.98518644928897...|       0.0|
|[1.0,71.0,78.0,50...|      0|[3.36564675734267...|[0.96661348896971...|       0.0|
|[1.0,73.0,50.0,10...|      0|[3.95026675618412...|[0.98111398151255...|       0.0|
|[1.0,73.0,50.0,10...|      0|[3.95026675618412...|[0.98111398151255...|       0.0|
|[1.0,77.0,56.0,30...|      0|[2.46501848770923...|[0.92165281002463...|       0.0|
|[1.0,79.0,60.0,42...|      0|[2.02913412571973...|[0.88382219905379...|       0.0|
|[1.0,79.0,80.0,25...|      0|[3.44401449875013...|[0.96905213799717...|       0.0|
|[1.0,80.0,74.0,11...|      0|[3.03113407439020...|[0.95396100648492...|    

In [ ]:
eval=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="Outcome")
eval.evaluate(model.transform(test))

0.8437468746874686

In [ ]:
# save model
model.save("model")

In [ ]:
# load saved model back to the environment
from pyspark.ml.classification import LogisticRegressionModel
model=LogisticRegressionModel.load("/content/model")

# TASK 7: Prediction on New Data with the saved model

In [ ]:
#create a new spark dataframe

testdf=spark.read.csv("/content/diabetes_dataset/new_test.csv",header=True,inferSchema=True)


In [ ]:
#print the schema
testdf.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



In [ ]:
#create an additional feature merged column 

test_data1=assembler.transform(testdf)

In [ ]:
#print the schema
test_data1.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#use model to make predictions
result=model.transform(test_data1)
result.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
#display the predictions
result.select("*").show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+--------------------+--------------------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|            features|       rawPrediction|         probability|prediction|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+--------------------+--------------------+----------+
|          1|    190|           78|           38|    150|45.1|                   0.153| 48|[1.0,190.0,78.0,3...|[-1.8779079829126...|[0.13262935178544...|       1.0|
|          0|     80|           84|           36|    120|50.2|                   0.211| 26|[0.0,80.0,84.0,36...|[2.02999214650948...|[0.88391027206397...|       0.0|
|          2|    138|           82|           46|    255|52.3|                   0.315| 30|[2.0,138.0,82.0,4...|[-0.5150233933851...|[0.37401666516610...|       1.0|
|   